In [1]:
import swat
import os
from sasctl import Session
from sasctl.tasks import register_model, publish_model
from sasctl.services import model_repository

# Connect to the CAS server

In [2]:
os.environ['CAS_CLIENT_SSL_CA_LIST'] = '/opt/sas/viya/config/etc/SASSecurityCertificateFramework/cacerts/trustedcerts.pem'
s = swat.CAS('frasepviya35smp.cloud.com', 5570)
# Define default session caslib
s.sessionprop.setsessopt(caslib='Public',metrics='true')

NOTE: 'Public' is now the active caslib.
NOTE: Action 'sessionProp.setSessOpt' used (Total process time):
NOTE:       real time               0.000280 seconds
NOTE:       cpu time                0.000257 seconds (91.79%)
NOTE:       total nodes             1 (16 cores)
NOTE:       total memory            125.75G
NOTE:       memory                  265.19K (0.00%)


+ Elapsed: 0.00028s, user: 0.000162s, sys: 9.5e-05s, mem: 0.259mb

In [3]:
# Name the existing CAS table to be used
indata = 'HMEQ_TRAIN'

# Declare python CASTable view on CAS in-memory table
castbl = s.CASTable(indata)

In [5]:
castbl.shape

(2381, 13)

In [6]:
castbl.describe()

,BAD,LOAN,MORTDUE,VALUE,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
count,2381.000000,2381.000000,2178.000000,2339.000000,2184.000000,2105.000000,2153.000000,2255.000000,2189.000000,2290.000000,1873.000000
mean,0.199916,18216.463671,73888.350849,102594.821005,8.851671,0.230404,0.449141,178.606762,1.172225,21.217467,33.473330
std,0.400021,10669.354029,43967.539209,58102.560140,7.520705,0.754975,1.094231,85.735499,1.721473,10.055845,8.228359
min,0.000000,1300.000000,2619.000000,9100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.524499
25%,0.000000,10800.000000,46057.000000,66410.000000,3.000000,0.000000,0.000000,115.136885,0.000000,15.000000,28.638063
50%,0.000000,16000.000000,64811.000000,89079.000000,7.000000,0.000000,0.000000,172.696685,1.000000,20.000000,34.644922
75%,0.000000,23000.000000,92487.000000,121382.000000,13.000000,0.000000,0.000000,229.177959,2.000000,26.000000,38.942760
max,1.000000,89000.000000,369874.000000,855909.000000,41.000000,10.000000,10.000000,1168.233561,14.000000,71.000000,143.949605


# Train a gradient boosting model to predict credit risk

In [8]:
s.loadactionset('decisionTree')

NOTE: Executing action 'builtins.loadActionSet'.
NOTE: Added action set 'decisionTree'.
NOTE: Action 'builtins.loadActionSet' used (Total process time):
NOTE:       real time               0.002008 seconds
NOTE:       cpu time                0.001985 seconds (98.85%)
NOTE:       total nodes             1 (16 cores)
NOTE:       total memory            125.75G
NOTE:       memory                  226.94K (0.00%)


[actionset]

 'decisionTree'

+ Elapsed: 0.00201s, user: 0.000961s, sys: 0.00102s, mem: 0.222mb

In [9]:
castbl.decisiontree.gbtreetrain(target='BAD',
                             inputs=['LOAN','MORTDUE','VALUE','YOJ','DEROG','DELINQ','CLAGE','NINQ','CLNO','DEBTINC'],
                             nominals=['BAD'],
                             savestate='gradboost_astore')

# Establish a reference to the newly created ASTORE table.
astore = s.CASTable('gradboost_astore')

NOTE: Executing action 'decisionTree.gbtreeTrain'.
NOTE: 937115 bytes were written to the table "gradboost_astore" in the caslib "Public".
NOTE: Action 'decisionTree.gbtreeTrain' used (Total process time):
NOTE:       real time               0.405143 seconds
NOTE:       cpu time                0.483555 seconds (119.35%)
NOTE:       total nodes             1 (16 cores)
NOTE:       total memory            125.75G
NOTE:       memory                  64.71M (0.05%)


# Train a random forest model to predict credit risk

In [10]:
castbl.decisionTree.forestTrain(
            inputs=['LOAN','MORTDUE','VALUE','YOJ','DEROG','DELINQ','CLAGE','NINQ','CLNO','DEBTINC'],
            ntrees=100,
            maxlevel=20,
            bootstrap=0.6,
            leafsize=5,
            nbins=50,
            vote='PROB',
            seed=12345,
            loh=0,
            quantilebin='TRUE',
            maxbranch=2,
            missing='USEINSEARCH',
            minuseinsearch=1,
            target="BAD",
            nominals=['BAD'],
            savestate={'name':'forest_astore','replace':'true'})

astore = s.CASTable('forest_astore')

NOTE: Executing action 'decisionTree.forestTrain'.
NOTE: 12834419 bytes were written to the table "forest_astore" in the caslib "Public".
NOTE: Action 'decisionTree.forestTrain' used (Total process time):
NOTE:       real time               1.104126 seconds
NOTE:       cpu time                3.703290 seconds (335.40%)
NOTE:       total nodes             1 (16 cores)
NOTE:       total memory            125.75G
NOTE:       memory                  170.62M (0.13%)


# Register the model in the central repository for versioning and deployment

In [11]:
with Session('frasepviya35smp.cloud.com', 'viyademo01', 'demopw'):
    model = register_model(astore, 'Random forest model', 'HMEQ_Modeling', force=True) 

NOTE: Executing action 'builtins.loadActionSet'.
NOTE: Added action set 'astore'.
NOTE: Action 'builtins.loadActionSet' used (Total process time):
NOTE:       real time               0.069495 seconds
NOTE:       cpu time                0.001479 seconds (2.13%)
NOTE:       total nodes             1 (16 cores)
NOTE:       total memory            125.75G
NOTE:       memory                  210.81K (0.00%)
NOTE: Executing action 'astore.describe'.
NOTE: Action 'astore.describe' used (Total process time):
NOTE:       real time               0.412548 seconds
NOTE:       cpu time                0.138908 seconds (33.67%)
NOTE:       total nodes             1 (16 cores)
NOTE:       total memory            125.75G
NOTE:       memory                  6.86M (0.01%)
NOTE: Executing action 'table.save'.
NOTE: Cloud Analytic Services saved the file _23C04832DE084716BA38B08A7.sashdat in caslib ModelStore.
NOTE: Action 'table.save' used (Total process time):
NOTE:       real time               0.247750

In [13]:
training_code_file  = open("/home/viyademo01/my_git/python_snippets/HMEQ_Train_RegisterMM_Astore_GBM_Model.ipynb", "r")

In [14]:
with Session('frasepviya35smp.cloud.com', 'viyademo01', 'demopw'):
    model = register_model(astore, 'Random forest model', 'HMEQ_Modeling', force=True)
    model_repository.add_model_content(model, file=training_code_file, name='traning_code.ipynb')

NOTE: Executing action 'builtins.loadActionSet'.
NOTE: Added action set 'astore'.
NOTE: Action 'builtins.loadActionSet' used (Total process time):
NOTE:       real time               0.000301 seconds
NOTE:       cpu time                0.000274 seconds (91.03%)
NOTE:       total nodes             1 (16 cores)
NOTE:       total memory            125.75G
NOTE:       memory                  209.81K (0.00%)
NOTE: Executing action 'astore.describe'.
NOTE: Action 'astore.describe' used (Total process time):
NOTE:       real time               0.136749 seconds
NOTE:       cpu time                0.136726 seconds (99.98%)
NOTE:       total nodes             1 (16 cores)
NOTE:       total memory            125.75G
NOTE:       memory                  6.86M (0.01%)
NOTE: Executing action 'table.save'.
NOTE: Cloud Analytic Services saved the file _92C6C2AFE99945369754C8C7E.sashdat in caslib ModelStore.
NOTE: Action 'table.save' used (Total process time):
NOTE:       real time               0.22998

# Optional : deploy the model on the SAS Viya realtime API engine for consumption in business applications

In [16]:
with Session('frasepviya35smp.cloud.com', 'viyademo01', 'demopw'):
    module = publish_model(model, 'maslocal',replace=1)

# Optional : use the API to score data in Python

In [18]:
with Session('frasepviya35smp.cloud.com', 'viyademo01', 'demopw'):
    response = module.score(LOAN=10,MORTDUE=1,VALUE=1,YOJ=10,DEROG=1,DELINQ=10,CLAGE=1,NINQ=1,CLNO=1,DEBTINC=1)

In [19]:
response

(0.5440674603174604, 0.4559325396825396, '1', '')

In [20]:
s.close()